In [11]:
#!/usr/bin/env python3
"""
verify_sshsig_standalone.py
---------------------------

Pure-Python verifier for OpenSSH detached signatures (“SSHSIG” files).

Usage
=====

    python verify_sshsig_standalone.py content.sig message.txt key.pub

If the signature is valid, it prints “✓ VALID”; otherwise “✗ FAILED”.
"""

from __future__ import annotations
import base64, hashlib, pathlib, struct, sys, textwrap
from typing import Tuple

# ═════════════════════════════════ SSH “string” helpers ════════════════════════
def read_ssh_string(buf: bytes, off: int = 0) -> Tuple[bytes, int]:
    """Decode one SSH binary ‘string’ (uint32 len || payload)."""
    if off + 4 > len(buf):
        raise ValueError("truncated SSH string length")
    (ln,) = struct.unpack(">I", buf[off : off + 4])
    start, end = off + 4, off + 4 + ln
    if end > len(buf):
        raise ValueError("truncated SSH string payload")
    return buf[start:end], end


def write_ssh_string(payload: bytes) -> bytes:
    return struct.pack(">I", len(payload)) + payload


# ════════════════════════ minimal Ed25519 verifier ════════════════════════════
# Adapted (condensed) from the public-domain ref10 code by Peter Schwabe et al.
# Only the *verify* path is kept; signing and scalar mult tables are stripped.

b = 256
q = 2**255 - 19
l = 2**252 + 27742317777372353535851937790883648493  # group order

def _inv(x):  # modular inverse via Fermat
    return pow(x, q - 2, q)

def _xrecover(y):
    xx = (y*y - 1) * _inv(d*y*y + 1) % q
    x  = pow(xx, (q+3)//8, q)
    if (x*x - xx) % q != 0:
        x = (x * I) % q
    if x % 2 != 0:
        x = q - x
    return x

def _ed_unpack(P: bytes):
    y = int.from_bytes(P, "little") & ((1 << 255) - 1)
    x = _xrecover(y)
    if bool(x & 1) != bool(P[31] >> 7):
        x = q - x
    return (x, y)

def _ed_add(P, Q):
    (x1, y1), (x2, y2) = P, Q
    x3 = (x1*y2 + x2*y1) * _inv(1 + d*x1*x2*y1*y2) % q
    y3 = (y1*y2 + x1*x2) * _inv(1 - d*x1*x2*y1*y2) % q
    return (x3, y3)

def _ed_double(P):
    return _ed_add(P, P)

# curve constants
d = -121665 * _inv(121666) % q
I = pow(2, (q - 1) // 4, q)
B = (
    15112221349535400772501151409588531511454012693041857206046113283949847762202,
    46316835694926478169428394003475163141307993866256225615783033603165251855960,
)

def _scalar_mult(P, e: int):
    Q = None
    for i in range(255, -1, -1):
        if Q is not None:
            Q = _ed_double(Q)
        if (e >> i) & 1:
            Q = P if Q is None else _ed_add(Q, P)
    return Q

def ed25519_verify(pub: bytes, msg: bytes, sig: bytes) -> bool:
    if len(pub) != 32 or len(sig) != 64:
        return False
    R_enc, S_enc = sig[:32], sig[32:]
    print(f"R: {R_enc.hex()}")
    print(f"S: {S_enc.hex()}")
    try:
        R = _ed_unpack(R_enc)
        A = _ed_unpack(pub)
    except ValueError:
        return False
    S = int.from_bytes(S_enc, "little")
    if S >= l:
        return False
    h = hashlib.sha512(R_enc + pub + msg).digest()
    h = int.from_bytes(h, "little") % l
    SB = _scalar_mult(B, S)
    R_plus_hA = _ed_add(R, _scalar_mult(A, h))
    return SB == R_plus_hA

import pathlib
import binascii

def pretty(val: bytes | int) -> str:
    """Render bytes as UTF-8 when printable, else as hex; leave ints as-is."""
    if isinstance(val, bytes):
        try:
            txt = val.decode("utf-8")
            if txt.isprintable():
                return f'"{txt}"'           # show printable bytes as text
        except UnicodeDecodeError:
            pass
        return "0x" + val.hex()             # otherwise hex-encode
    return str(val)

# ═════════════════════════ SSHSIG parsing & verification ═══════════════════════
def parse_sshsig(path: pathlib.Path):
    """Return dict with all interesting pieces extracted from *path*."""
    armour = [ln.strip() for ln in path.read_text().splitlines() if ln.strip()]
    body_b64 = "".join(ln for ln in armour if not ln.startswith("-----"))
    blob = base64.b64decode(body_b64)

    if not blob.startswith(b"SSHSIG"):
        raise ValueError("Not an SSHSIG blob")
    off = 6
    version, = struct.unpack(">I", blob[off:off+4]); off += 4
    pk_raw,  off = read_ssh_string(blob, off)
    ns_raw,  off = read_ssh_string(blob, off)
    resvd,   off = read_ssh_string(blob, off)
    hash_alg,off = read_ssh_string(blob, off)
    sig_raw, off = read_ssh_string(blob, off)
    if off != len(blob):
        raise ValueError("Trailing bytes in SSHSIG")

    # break out embedded pk / sig
    pk_alg, pos = read_ssh_string(pk_raw, 0)
    pk_bytes, _ = read_ssh_string(pk_raw, pos)
    sig_alg, pos = read_ssh_string(sig_raw, 0)
    sig_bytes, _ = read_ssh_string(sig_raw, pos)

    parsed = {
        "version": version,
        "namespace": ns_raw,
        "hash_alg": hash_alg,
        "pk_alg": pk_alg,
        "pk_bytes": pk_bytes,
        "sig_alg": sig_alg,
        "sig_bytes": sig_bytes,
    }

    print("== SSHSIG fields ==")
    for key, value in parsed.items():
        # skip the "_raw" sub-dict if you included it in your parse() return
        if key == "_raw":
            continue
        print(f"{key:10}: {pretty(value)}")

    return parsed


def verify_sshsig(sig_path: pathlib.Path,
                  msg_path: pathlib.Path) -> bool:
    parsed = parse_sshsig(sig_path)
    msg = msg_path.read_bytes()

    # step-1: recompute digest(message) with the declared hash
    hname = parsed["hash_alg"].decode()
    if hname not in ("sha512", "sha256"):
        raise ValueError(f"unsupported hash {hname}")
    digest = hashlib.new(hname, msg).digest()

    # step-2: rebuild the wrapper that got signed
    wrapper = (
        b"SSHSIG"
        + write_ssh_string(parsed["namespace"])
        + write_ssh_string(b"")               # reserved
        + write_ssh_string(parsed["hash_alg"])
        + write_ssh_string(digest)
    )

    # step-3: verify according to algorithm
    if parsed["sig_alg"] != b"ssh-ed25519" or parsed["pk_alg"] != b"ssh-ed25519":
        raise ValueError("this standalone verifier only supports Ed25519")
    return ed25519_verify(parsed["pk_bytes"], wrapper, parsed["sig_bytes"])


# ═══════════════════════════════════ main ══════════════════════════════════════
if __name__ == "__main__":
    sig_file  = pathlib.Path("content.sig")
    msg_file  = pathlib.Path("message.txt")

    ok = verify_sshsig(sig_file, msg_file)
    print("✓ VALID" if ok else "✗ FAILED")


== SSHSIG fields ==
version   : 1
namespace : "file"
hash_alg  : "sha512"
pk_alg    : "ssh-ed25519"
pk_bytes  : 0x9d65ed157550e00c3d094fdcc8fd747659c61b5f5adce10c055b7c84fe7e615b
sig_alg   : "ssh-ed25519"
sig_bytes : 0x3524cd973837560ae9dfc3fe6fe118d28f5922f928c0b9cfe8ae5ff88c94b9a0818139b00915a015cb5d91097a0382afe8d043cd6b88adffb7447c844cddab06
R: 3524cd973837560ae9dfc3fe6fe118d28f5922f928c0b9cfe8ae5ff88c94b9a0
S: 818139b00915a015cb5d91097a0382afe8d043cd6b88adffb7447c844cddab06
✓ VALID
